<img src="datamecum_logo.png" align="right" style="float" width="400">
<font color="#CA3532"><h1 align="left">Programa técnico intensivo en data science. Datamecum.</h1></font>
<font color="#6E6E6E"><h2 align="left">Módulo Implementación de Sistemas Data-driven.</h2></font> 
<font color="#6E6E6E"><h2 align="left">Tarea 1. Pipeline entrenamiento de modelos.</h2></font> 

#### Daniel Ruiz Riquelme

##  Install dependencies

In [1]:
!pip install metaflow
!pip install scikit-learn
!pip install pandas

## Set username

In [2]:
# Set username for workflows
import os
os.environ["USERNAME"] = "daniel"

In [3]:
%%writefile metaflow_trainingflow.py
from metaflow import FlowSpec, Parameter, step

class TrainingFlow(FlowSpec):
    # Define the parameters for the flow
    max_depth = Parameter('max_depth', default=2, help='Max depth of the random forest classifier')
    n_estimators = Parameter('n_estimators', default=100, help='Number of estimators for the random forest classifier')
    random_state = Parameter('random_state', default=0, help='Random state for the random forest classifier')
    
    @step
    def start(self):
        # Start the flow
        self.next(self.ingest_data)
        
    @step
    def ingest_data(self):
        from sklearn.datasets import load_iris
    
        # Load the iris dataset
        iris = load_iris()
        
        #pylint: disable=no-member
        self.X = iris.data
        self.y = iris.target
        #pylint: enable=no-member
        
        self.next(self.split_data)

    @step
    def split_data(self):
        #Split the data into train and test
        from sklearn.model_selection import train_test_split
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2)
        self.next(self.train)

    @step
    def train(self):
        from sklearn.ensemble import RandomForestClassifier
        # Train the model
        self.clf = RandomForestClassifier(max_depth=self.max_depth, n_estimators=self.n_estimators, random_state=self.random_state)
        self.clf.fit(self.X_train, self.y_train)
        self.next(self.show_metrics)

    @step
    def show_metrics(self):
        from sklearn.metrics import classification_report, confusion_matrix
        # Print some metrics
        y_pred = self.clf.predict(self.X_test)
        print(confusion_matrix(self.y_test, y_pred))
        print(classification_report(self.y_test, y_pred))
        self.next(self.register_model)
        
    @step
    def register_model(self):
        # Save the model
        import pickle
        with open('model.pkl', 'wb') as f:
            pickle.dump(self.clf, f)
                 
        self.next(self.end)
        
    @step
    def end(self):
        pass
    
if __name__ == '__main__':
    TrainingFlow()

Overwriting metaflow_trainingflow.py


In [4]:
!python metaflow_trainingflow.py run --max_depth 2 --n_estimators 100 --random_state 0

Metaflow 2.7.22 executing TrainingFlow for user:daniel
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2023-02-20 20:07:27.102 Workflow starting (run-id 1676920046615427):
2023-02-20 20:07:27.282 [1676920046615427/start/1 (pid 1188)] Task is starting.
2023-02-20 20:07:28.728 [1676920046615427/start/1 (pid 1188)] Task finished successfully.
2023-02-20 20:07:28.917 [1676920046615427/ingest_data/2 (pid 1194)] Task is starting.
2023-02-20 20:07:31.639 [1676920046615427/ingest_data/2 (pid 1194)] Task finished successfully.
2023-02-20 20:07:31.896 [1676920046615427/split_data/3 (pid 1225)] Task is starting.
2023-02-20 20:07:34.144 [1676920046615427/split_data/3 (pid 1225)] Task finished successfully.
2023-02-20 20:07:34.376 [1676920046615427/train/4 (pid 1253)] Task is starting.
2023-02-20 20:07:37.202 [1676920046615427/train/4 (pid 1253)] Task finished successfully.
2023-02-20 20:07:37.444 [1676920046615427/show_metrics/5 (pid 1281)] Task is starting